In [1]:
#initial imports
import pandas as pd

LTPATH = r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\bonds\concatenated_1823.xlsx"
DTPATH = r""


In [2]:
#clean demographic data
# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\demog\cc-est2022-all.csv'

# Read the Excel file into a pandas DataFrame
demog = pd.read_csv(excel_file, encoding='latin1')

# Display the columns
print("Columns in the Excel file:")
for column in demog.columns:
    print(column)

Columns in the Excel file:
SUMLEV
STATE
COUNTY
STNAME
CTYNAME
YEAR
AGEGRP
TOT_POP
TOT_MALE
TOT_FEMALE
WA_MALE
WA_FEMALE
BA_MALE
BA_FEMALE
IA_MALE
IA_FEMALE
AA_MALE
AA_FEMALE
NA_MALE
NA_FEMALE
TOM_MALE
TOM_FEMALE
WAC_MALE
WAC_FEMALE
BAC_MALE
BAC_FEMALE
IAC_MALE
IAC_FEMALE
AAC_MALE
AAC_FEMALE
NAC_MALE
NAC_FEMALE
NH_MALE
NH_FEMALE
NHWA_MALE
NHWA_FEMALE
NHBA_MALE
NHBA_FEMALE
NHIA_MALE
NHIA_FEMALE
NHAA_MALE
NHAA_FEMALE
NHNA_MALE
NHNA_FEMALE
NHTOM_MALE
NHTOM_FEMALE
NHWAC_MALE
NHWAC_FEMALE
NHBAC_MALE
NHBAC_FEMALE
NHIAC_MALE
NHIAC_FEMALE
NHAAC_MALE
NHAAC_FEMALE
NHNAC_MALE
NHNAC_FEMALE
H_MALE
H_FEMALE
HWA_MALE
HWA_FEMALE
HBA_MALE
HBA_FEMALE
HIA_MALE
HIA_FEMALE
HAA_MALE
HAA_FEMALE
HNA_MALE
HNA_FEMALE
HTOM_MALE
HTOM_FEMALE
HWAC_MALE
HWAC_FEMALE
HBAC_MALE
HBAC_FEMALE
HIAC_MALE
HIAC_FEMALE
HAAC_MALE
HAAC_FEMALE
HNAC_MALE
HNAC_FEMALE


In [3]:
demog = demog[demog['AGEGRP'] == 0]
demog = demog[demog['YEAR'] == 1]

demog['CTYNAME'] = demog['CTYNAME'].apply(lambda x: x[:-7] if x.endswith(" County") else x)

demog['PropWhiteTC'] = (demog['WAC_MALE'] + demog['WAC_FEMALE']) / demog['TOT_POP']
demog['PropBlackTC'] = (demog['BAC_MALE'] + demog['BAC_FEMALE']) / demog['TOT_POP']
demog['PropAsianTC'] = (demog['AAC_MALE'] + demog['AAC_FEMALE']) / demog['TOT_POP']
demog['PropHispaTC'] = (demog['H_MALE'] + demog['H_FEMALE']) / demog['TOT_POP']
demog['PropWhiteTA'] = (demog['WA_MALE'] + demog['WA_FEMALE']) / demog['TOT_POP']
demog['PropBlackTA'] = (demog['BA_MALE'] + demog['BA_FEMALE']) / demog['TOT_POP']
demog['PropAsianTA'] = (demog['AA_MALE'] + demog['AA_FEMALE']) / demog['TOT_POP']

demog = demog.loc[:, ['CTYNAME', 'STNAME', 'STATE', 'COUNTY','PropWhiteTC',"TOT_POP"
                      'PropBlackTC', 'PropAsianTC', 'PropHispaTC',
                      'PropWhiteTA', 'PropAsianTA', 'PropBlackTA']]

demog.head()


,CTYNAME,STNAME,STATE,COUNTY,PropWhiteTC,PropBlackTC,PropAsianTC,PropHispaTC,PropWhiteTA,PropAsianTA,PropBlackTA
0,Autauga,Alabama,1,1,0.776742,0.214500,0.016921,0.031342,0.759107,0.011751,0.204466
76,Baldwin,Alabama,1,3,0.891582,0.096440,0.015943,0.047105,0.872705,0.011063,0.087405
152,Barbour,Alabama,1,5,0.501943,0.491635,0.007057,0.047613,0.489613,0.004599,0.481367
228,Bibb,Alabama,1,7,0.779776,0.217848,0.004709,0.028834,0.767444,0.002601,0.210404
304,Blount,Alabama,1,9,0.970455,0.022577,0.006021,0.095890,0.957128,0.003754,0.017216


In [4]:
#merge and clean bond data

# Replace 'your_excel_file.xlsx' with the path to your Excel file

excel_file = r'C:\Users\rra3\Desktop\summer2024TYP\muni\data\bonds\concatenated_1823.xlsx'
# Read the Excel file into a pandas DataFrame
muni = pd.read_excel(excel_file)

# Display the columns
print("Columns in the Excel file:")
for column in demog.columns:
    print(column)

Columns in the Excel file:
CTYNAME
STNAME
STATE
COUNTY
PropWhiteTC
PropBlackTC
PropAsianTC
PropHispaTC
PropWhiteTA
PropAsianTA
PropBlackTA


In [5]:
muni = muni.rename(columns={"State of Issuer": "STNAME"})

# Create a new DataFrame to hold the expanded rows
expanded_rows = []

# Iterate through each row in muni DataFrame
for index, row in muni.iterrows():
    # Initialize the status variable
    status = 0  # 0 means no change
    
    # Check if 'County' is a string and contains a slash
    if isinstance(row['County'], str) and '/' in row['County']:
        # Split the county names separated by a slash
        counties = row['County'].split('/')
        # Create a new row for each county
        for county in counties:
            new_row = row.copy()  # Create a copy of the original row
            new_row['County'] = county.strip()  # Update the 'County' column with the individual county
            expanded_rows.append(new_row)  # Append the new row to the list of expanded rows
        status = 1  # 1 means split
    elif isinstance(row['County'], float):
        status = 2  # 2 means it's a float
    
    # Add the status to the row
    row['Status'] = status
    
    # Append the row to the list of expanded rows
    expanded_rows.append(row)

# Create a new DataFrame from the list of expanded rows
expanded_muni = pd.DataFrame(expanded_rows)

muni = expanded_muni.rename(columns={"County": "CTYNAME"})

In [6]:
muni.head()

muni.rename(columns={"Fitch's Insured Long Term Rating": 'Fitch_Long', "Fitch's Insured Short Term Rating": 'Fitch_Short', "Moody's Insured Long Term Rating": 'Moody_Long', "Moody's Insured Short Term Rating": 'Moody_Short'}, inplace=True)

# Define a function to extract the highest Moody's rating from a string
def extract_highest_rating(s):
    ratings_order = ['Aaa', 'Aa1', 'Aa2', 'Aa3', 'A1', 'A2', 'A3', 
                     'Baa1', 'Baa2', 'Baa3', 'Ba1', 'Ba2', 'Ba3',
                     'Caa1', 'Caa2', 'Caa3', 'Ca', 'C','NR']
    max_rating = 'C'  # Default lowest rating
    for rating in ratings_order:
        if rating in s:
            max_rating = rating
            break  # Stop when the highest rating is found
    return max_rating


# Define a function to assign an ordinal number to each rating
def rating_to_number(rating):
    rating_to_num = {'Aaa': 1, 'Aa1': 2, 'Aa2': 3, 'Aa3': 4, 'A1': 5, 'A2': 6, 'A3': 7,
                     'Baa1': 8, 'Baa2': 9, 'Baa3': 10, 'Ba1': 11, 'Ba2': 12, 'Ba3': 13,
                     'Caa1': 14, 'Caa2': 15, 'Caa3': 16, 'Ca': 17, 'C': 18}
    return rating_to_num.get(rating, 18)  # Default value is 18 if rating not found

# Apply the functions to each row and create new columns 'Moody_High' and 'Moody_Number'
muni['Moody_High'] = muni['Moody_Long'].fillna('').apply(extract_highest_rating)
muni['Moody_Number'] = muni['Moody_High'].apply(rating_to_number)




print("Columns in the Excel file:")
for column in muni.columns:
    print(column)



Columns in the Excel file:
Sale Date
Date at Maturity
Date at Maturity.1
Sale Date.1
Date of Maturities (Maturity Level)
Issuer's CUSIP
9-digit CUSIP
9-digit CUSIP.1
Issuer (Full Name)
General Use of proceeds (main)
Security
Tax Status
CTYNAME
STNAME
Coupon of Any Term Maturity or First Serial Maturity
Coupon Type
Final maturity yield
Callable Issue
Credit Enhancer Type
Credit Enhancer Type.1
Rule 144A Flag (Y/N)
Tax Status.1
General Use of proceeds (main).1
Years to Maturity
Fitch_Long
Fitch_Short
Moody_Long
Moody_Short
Underwriters' fee (USD)
Issuer (Full Name).1
Issuer Type Description
Charter School Flag (Y/N)
Climate Bond (Y/N)
Environmental Social and Governance Bond Type
Green Bond Flag (Y/N)
Offering Type
Par Amount (USD Millions)
Pooled
Project
Status
Moody_High
Moody_Number


In [7]:
merged_df = pd.merge(muni, demog, on=['STNAME', 'CTYNAME'], how='left')

In [8]:
merged_df.to_stata(r"C:\Users\rra3\Desktop\summer2024TYP\muni\data\stata\MuniDemo.dta",version=117)

C:\Users\rra3\AppData\Local\Temp\ipykernel_33200\1548106995.py:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Sale Date   ->   Sale_Date
    Date at Maturity   ->   Date_at_Maturity
    Date at Maturity.1   ->   Date_at_Maturity_1
    Sale Date.1   ->   Sale_Date_1
    Date of Maturities (Maturity Level)   ->   Date_of_Maturities__Maturity_Lev
    Issuer's CUSIP   ->   Issuer_s_CUSIP
    9-digit CUSIP   ->   _9_digit_CUSIP
    9-digit CUSIP.1   ->   _9_digit_CUSIP_1
    Issuer (Full Name)   ->   Issuer__Full_Name_
    General Use of proceeds (main)   ->   General_Use_of_proceeds__main_
    Tax Status   ->   Tax_Status
    Coupon of Any Term Maturity or First Serial Maturity   ->   Coupon_of_Any_Term_Maturity_or_F
    Coupon Type   ->   Coupon_Type
    Final maturity yield   ->   Final_maturity_yield
    Callable Issue   ->   Callable_Issue
    Credit Enhancer Type   ->   Credit_Enhancer_Type
    Credit